<a href="https://colab.research.google.com/github/yanil-03/python/blob/main/recap(std%2Cencoding%2C_transformer).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Normal Distribution ---- > Plot central tendancy (mean , median , mode) ---- >
# left side and right side equal data points .

# Benifit ---- > Data centroid --- > calculations easy --- > model training time less , solution time fast .


In [3]:
# (1). Standardization ---- > mean =0 , std = 1
# (2). Normalization ---- > min_value = 0 , max_value = 1
# (3). Encoding ---- > Convert our categorical data into Numerical data .
# Machine -- Numbers --- > texts xx ?

# (a). LabelEncoding ---- > gender (Male , female) --- > Male-1 , female-0 , one-dimensionl data .
# (b). Ordinal Encoding --- > gender(Male , Female) --- > We can select our sub-category on a position and
# it supports multi-dimensional data also .
# (c). OneHotEncoding --- > gender(Male , Female) ---- > gender --- > Male_gender , Female_gender --- >
# but it provides only values not column name .
# (d). get_dummies ----> gender(Male , Female) ---- > gender_m , gender_f --- >
#  but it provides only values not column name .

In [4]:
# (4). Column transformer ----- > container --- > (constant , approach_name , column_name)

# Data --- > covid_toy.csv ---- > 5 columns --- > 1-1 column numerical data convert --- >
# In last , I will add all individual column and check final data shape .

In [5]:
import numpy as np
import pandas as pd


In [7]:
df = pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [11]:
x = df.drop(columns=['has_covid'])
y = df['has_covid']

In [12]:
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [13]:
x_train.head(2)

,age,gender,fever,cough,city
55,81,Female,101.0,Mild,Mumbai
88,5,Female,100.0,Mild,Kolkata


In [14]:
# manually type output

In [16]:
# adding simpleImputer to fever column
si = SimpleImputer(strategy="mean")
x_train_fever = si.fit_transform(x_train[['fever']])

# also the test data
x_test_fever = si.fit_transform(x_test[['fever']])

x_train_fever.shape

(80, 1)

In [17]:
# Ordinal Encoding ---> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])

x_train_cough = oe.fit_transform(x_train[['cough']])

# also test data
x_test_cough = oe.fit_transform(x_test[['cough']])

x_train_cough.shape

(80, 1)

In [19]:
# OneHot encoding ---> gender, city
ohe = OneHotEncoder(drop='first',sparse_output=False)
x_train_gender_city = ohe.fit_transform(x_train[['gender','city']])

# also the test data
x_test_gender_city = ohe.fit_transform(x_test[['gender','city']])

x_train_gender_city.shape

(80, 4)

In [20]:
x_train_age = x_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
x_test_age = x_test.drop(columns=['gender','fever','cough','city']).values

x_train_age.shape

(80, 1)

In [21]:
# x_train_cough ----> (80,1)
# x_train_gender_city ----> (80,4)
# x_train_fever ----> (80,1)
# x_train_age ----> (80,1)

In [22]:
x_train_transformed = np.concatenate((x_train_age,
                                      x_train_fever,
                                      x_train_gender_city,
                                      x_train_cough) , axis=1)

In [23]:
x_train_transformed.shape

(80, 7)

In [24]:
# NOW using column Transformer


In [26]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),  # in a 'fever' column by the help of SI
    # we fill missing value by mean , meadian , mode
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']), # by this
    # process we encode our data


    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')    # remainder = passthrough --->
# it means rest all the columns reamin same.

In [30]:
tr_sc = transformer.fit_transform(x_train)
# x_test_transformed = transformer.transform(x_test)|

In [29]:
transformer

ColumnTransformer(remainder='passthrough',
                  transformers=[('tnf1', SimpleImputer(), ['fever']),
                                ('tnf2',
                                 OrdinalEncoder(categories=[['Mild',
                                                             'Strong']]),
                                 ['cough']),
                                ('tnf3',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['gender', 'city'])])

In [31]:
tr_sc.shape

(80, 7)

In [32]:
df.head(2)

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes


In [33]:
df.isnull().sum()

,0
age,0
gender,0
fever,10
cough,0
city,0
has_covid,0


In [34]:
si = SimpleImputer()

df['fever'] = si.fit_transform(df[['fever']])

In [35]:
df.isnull().sum()

,0
age,0
gender,0
fever,0
cough,0
city,0
has_covid,0


In [36]:
from sklearn.preprocessing import LabelEncoder

In [40]:
lb = LabelEncoder()


In [41]:
df['gender'] = lb.fit_transform(df['gender'])
df['cough']  = lb.fit_transform(df['cough'])
df['city']  = lb.fit_transform(df['city'])
df['has_covid']  = lb.fit_transform(df['has_covid'])


In [42]:
np.round(df.describe(), 1)

,age,gender,fever,cough,city,has_covid
count,100.0,100.0,100.0,100.0,100.0,100.0
mean,44.2,0.4,100.8,0.4,1.3,0.4
std,24.9,0.5,1.9,0.5,1.1,0.5
min,5.0,0.0,98.0,0.0,0.0,0.0
25%,20.0,0.0,99.0,0.0,0.0,0.0
50%,45.0,0.0,100.9,0.0,1.0,0.0
75%,66.5,1.0,102.0,1.0,2.0,1.0
max,84.0,1.0,104.0,1.0,3.0,1.0


In [44]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [45]:
df_sc = sc.fit_transform(df)

In [46]:
df_new = pd.DataFrame(df_sc,columns=df.columns)

In [48]:
np.round(df_new.describe(),2)

,age,gender,fever,cough,city,has_covid
count,100.00,100.00,100.00,100.00,100.00,100.00
mean,0.00,0.00,-0.00,0.00,-0.00,0.00
std,1.01,1.01,1.01,1.01,1.01,1.01
min,-1.58,-0.83,-1.47,-0.78,-1.25,-0.90
25%,-0.98,-0.83,-0.95,-0.78,-1.25,-0.90
50%,0.03,-0.83,0.04,-0.78,-0.32,-0.90
75%,0.90,1.20,0.60,1.28,0.62,1.11
max,1.61,1.20,1.63,1.28,1.55,1.11
